# 基于pyecharts的可交互可视化(pyecharts)

1. matplotlib：纯python出图，可批量出图，缺点是出的图片为静态图片无法交互
2. folium：主要功能是绘制地图，javascript出图可交互，坐标系为wgs84，数据不需要转坐标
3. echarts：可绘制各种图表，也能绘制地图，javascript出图可交互，但绘制地图时底图一般采用百度地图，需要转坐标系

## 数据获取
很多网站都提供了疫情分布情况，数据都是公开的直接抓就行。这里以腾讯新闻的疫情发布链接为例，观察网络链接可以找到数据获取的访问请求

In [3]:
import urllib #urllib的request模块可以非常方便地抓取URL内容，也就是发送一个GET请求到指定的页面，然后返回HTTP的响应
import json
url = 'https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5'
request = urllib.request.Request(url)
response = urllib.request.urlopen(request)
datajson=json.loads(response.read().decode('utf8'))
datajson=json.loads(datajson['data'])

In [4]:
#提取各省份的数据
import pandas as pd
provincedata = pd.DataFrame(datajson['areaTree'][0]['children'])
provincedata.head(5)

,name,today,total,children
0,香港,"{'confirm': 14, 'confirmCuts': 0, 'isUpdated':...","{'nowConfirm': 235, 'confirm': 4971, 'suspect'...","[{'name': '地区待确认', 'today': {'confirm': 14, 'c..."
1,上海,"{'confirm': 1, 'confirmCuts': 0, 'isUpdated': ...","{'nowConfirm': 41, 'confirm': 948, 'suspect': ...","[{'name': '境外输入', 'today': {'confirm': 1, 'con..."
2,广东,"{'confirm': 4, 'confirmCuts': 0, 'isUpdated': ...","{'nowConfirm': 21, 'confirm': 1782, 'suspect':...","[{'name': '境外输入', 'today': {'confirm': 4, 'con..."
3,台湾,"{'confirm': 1, 'confirmCuts': 0, 'isUpdated': ...","{'nowConfirm': 16, 'confirm': 499, 'suspect': ...","[{'name': '地区待确认', 'today': {'confirm': 1, 'co..."
4,四川,"{'confirm': 1, 'confirmCuts': 0, 'isUpdated': ...","{'nowConfirm': 15, 'confirm': 666, 'suspect': ...","[{'name': '境外输入', 'today': {'confirm': 1, 'con..."


In [5]:
#整理一下数据，把total里面的数据展开
data1 = pd.DataFrame(list(provincedata['total']))
data1['name'] = provincedata['name']
data1.head(5)

,nowConfirm,confirm,suspect,dead,deadRate,showRate,heal,healRate,showHeal,name
0,235,4971,0,101,2.03,False,4635,93.24,True,香港
1,41,948,0,7,0.74,False,900,94.94,True,上海
2,21,1782,0,8,0.45,False,1753,98.37,True,广东
3,16,499,0,7,1.40,False,476,95.39,True,台湾
4,15,666,0,3,0.45,False,648,97.30,True,四川


首先把数据整理成echarts认识的格式,就是如下:

In [6]:
data1[['name','confirm']].values

array([['香港', 4971],
       ['上海', 948],
       ['广东', 1782],
       ['台湾', 499],
       ['四川', 666],
       ['福建', 390],
       ['山东', 831],
       ['陕西', 381],
       ['浙江', 1281],
       ['云南', 204],
       ['天津', 234],
       ['河北', 365],
       ['广西', 258],
       ['江苏', 665],
       ['重庆', 584],
       ['内蒙古', 261],
       ['河南', 1277],
       ['辽宁', 264],
       ['山西', 203],
       ['湖北', 68139],
       ['安徽', 991],
       ['澳门', 46],
       ['海南', 171],
       ['江西', 935],
       ['北京', 935],
       ['宁夏', 75],
       ['新疆', 902],
       ['甘肃', 170],
       ['青海', 18],
       ['黑龙江', 948],
       ['贵州', 147],
       ['湖南', 1019],
       ['西藏', 1],
       ['吉林', 157]], dtype=object)

In [10]:
from pyecharts import options as opts
from pyecharts.charts import Map
#创建echarts对象c
c = (
    Map()#告诉echarts这个是Map形式的图表
    .add("确诊",data1[['name','confirm']].values,"china")#加一个数据，这个数据名叫”确诊“，数据的地图是echarts自带的china
    .set_global_opts(
        #对图表添加设置
        title_opts = opts.TitleOpts(title = '疫情地图')
    )
)
c.render('疫情地图.html')

/home/liu/miniconda3/lib/python3.7/site-packages/pyecharts/charts/chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


'/home/liu/Documents/jupyter/用Python分析时空数据的教程/可交互可视化/疫情地图.html'

In [16]:
from pyecharts import options as opts
from pyecharts.charts import Map
c = (
    Map()
    .add(
        "确诊",
        data1[['name','confirm']].values,
        "china",
        is_roam = False, #不可鼠标缩放和平移漫游
        zoom = 1.2, #当前视角的缩放比例
        is_map_symbol_show = False, #是否显示标记图形
        label_opts = opts.LabelOpts(position = 'inside')#标签尽量放在图形区域内
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title='疫情地图'),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=True,#设定分段颜色显示
                                          pieces=[{'min': 10000,'label':'10000人以上'}, #设定分段的值
                                                  {'min': 1000, 'max': 9999,'label':'1000-9999人'},
                                                  {'min': 500, 'max': 999,'label':'500-999人'},
                                                  {'min': 100, 'max': 499,'label':'100-499人'},
                                                  {'min': 10, 'max': 99,'label':'10-99人'},
                                                  {'min': 1, 'max': 9,'label':'1-9人'}],
                                           range_color=["#b4e0f3","#70b4eb","#1482e5","#1c3fbf","#070093" ] #调整显示颜色
                                         ),
        
    )
)
c.render('疫情地图.html')

/home/liu/miniconda3/lib/python3.7/site-packages/pyecharts/charts/chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


'/home/liu/Documents/jupyter/用Python分析时空数据的教程/可交互可视化/疫情地图.html'

## 单个省份数据可视化

In [18]:
import pandas as pd
#提取省份的数据
province = '山东'
shandongdata = pd.DataFrame(provincedata[provincedata['name'] == province]['children'].iloc[0])
#整理数据,将total里面的数据展开
data2 = pd.DataFrame(list(shandongdata['total']))
data2['name'] = shandongdata['name'] + '市'
data2.head(5)

,nowConfirm,confirm,suspect,dead,deadRate,showRate,heal,healRate,showHeal,name
0,11,68,0,0,0.00,False,57,83.82,True,境外输入市
1,0,44,0,0,0.00,False,44,100.00,True,潍坊市
2,0,260,0,0,0.00,False,260,100.00,True,济宁市
3,0,47,0,0,0.00,False,47,100.00,True,济南市
4,0,49,0,0,0.00,False,49,100.00,True,临沂市


In [24]:
from pyecharts import options as opts
from pyecharts.charts import Map
c = (
    Map()
    .add("确诊", 
         data2[['name','confirm']].values, 
         province,
         is_roam = False,#不可鼠标缩放和平移漫游
         zoom = 1.2,#当前视角的缩放比例
         is_map_symbol_show = False, # 是否显示标记图形
         label_opts = opts.LabelOpts(position = 'inside'),#标签尽量放在图形区域内
        )
    .set_global_opts(
        title_opts=opts.TitleOpts(title=province+'疫情地图'),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=True,#设定分段颜色显示
                                          pieces=[{'min': 200, 'label':'200人以上'},#设定分段的值
                                                  {'min': 100, 'max': 199,'label':'100-199人'},
                                                  {'min': 50, 'max': 99,'label':'50-99人'},
                                                  {'min': 10, 'max': 49,'label':'10-49人'},
                                                  {'min': 1, 'max': 9,'label':'1-9人'}],
                                           range_color=["#b4e0f3","#70b4eb","#1482e5","#1c3fbf","#070093" ] #调整显示颜色
                                         ),
    )
)
c.render(province+'疫情地图.html')

/home/liu/miniconda3/lib/python3.7/site-packages/pyecharts/charts/chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


'/home/liu/Documents/jupyter/用Python分析时空数据的教程/可交互可视化/山东疫情地图.html'

In [1]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [2]:
pip show --verbose pandas

ERROR: Exception:
Traceback (most recent call last):
  File "/home/liu/miniconda3/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/home/liu/miniconda3/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/liu/miniconda3/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3012, in _parsed_pkg_info
    return self._pkg_info
  File "/home/liu/miniconda3/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/liu/miniconda3/lib/python3.7/site-packages

In [3]:
pip install -U pandas

     |████████████████████████████████| 10.5 MB 33 kB/s eta 0:00:01     |██████████████████████████▌     | 8.7 MB 24 kB/s eta 0:01:14
ERROR: Could not install packages due to an EnvironmentError: [Errno 2] 没有那个文件或目录: '/home/liu/miniconda3/lib/python3.7/site-packages/pytz-2020.1.dist-info/METADATA'

Note: you may need to restart the kernel to use updated packages.
